In [1]:
import requests
import folium
import json
import os
from geopy.geocoders import Nominatim
from dotenv import load_dotenv

In [2]:
load_dotenv("../.env")

True

In [3]:
def get_current_location():
    """
    Attempts to get the current location based on the user's IP address.
    Falls back to manual input if automatic detection fails.
    """
    try:
        response = requests.get('https://ipinfo.io/json')
        data = response.json()
        loc = data['loc'].split(',')
        latitude = float(loc[0])
        longitude = float(loc[1])
        print(f"Detected current location: {data['city']}, {data['region']}, {data['country']}")
        return (latitude, longitude)
    except Exception as e:
        print("Could not automatically detect location.")
        latitude = float(input("Enter your current latitude: "))
        longitude = float(input("Enter your current longitude: "))
        return (latitude, longitude)

In [4]:
get_current_location()

Detected current location: Hanoi, Hanoi, VN


(21.0245, 105.8412)

In [5]:
def get_destination(destination: str = None, **kwargs):
    """
    Prompts the user to enter the destination address and returns its coordinates.
    """
    assert destination is not None, "Destination address must be provided."

    geolocator = Nominatim(user_agent="route_planner")
    try:
        location = geolocator.geocode(destination)
        print(f"Destination location: {location.address}")
        return (location.latitude, location.longitude)
    except Exception as e:
        print("Could not find the destination address.")
        return None

In [6]:
get_destination(destination="Da Lat")

Destination location: Thành phố Đà Lạt, Tỉnh Lâm Đồng, Việt Nam


(11.9402416, 108.4375758)

In [7]:
def calculate_route(start_coords, end_coords, api_key = os.getenv("ORS_TOKEN")):
    """
    Uses OpenRouteService API to calculate the route between two coordinates.
    Returns the route geometry and the distance in kilometers.
    """
    url = 'https://api.openrouteservice.org/v2/directions/driving-car/geojson'
    headers = {
        'Authorization': api_key,
        'Content-Type': 'application/json'
    }
    body = {
        "coordinates": [
            [start_coords[1], start_coords[0]],  # [lng, lat]
            [end_coords[1], end_coords[0]]
        ]
    }
    response = requests.post(url, headers=headers, json=body)
    if response.status_code == 200:
        data = response.json()
        distance = data['features'][0]['properties']['segments'][0]['distance'] / 1000
        geometry = data['features'][0]['geometry']['coordinates']
        route = [(coord[1], coord[0]) for coord in geometry]
        return route, distance
    else:
        print(f"Error fetching route: {response.status_code} - {response.text}")
        return None, None

In [8]:
calculate_route(get_current_location(), get_destination("Da Lat"))[1]

Detected current location: Hanoi, Hanoi, VN
Destination location: Thành phố Đà Lạt, Tỉnh Lâm Đồng, Việt Nam


1409.4773

In [11]:
def plot_route(route, start_coords, end_coords):
    """
    Plots the route on an interactive map and saves it as an HTML file.
    """
    # Initialize the map at the starting point
    m = folium.Map(location=start_coords, zoom_start=13)

    # Add the route as a PolyLine
    folium.PolyLine(route, color="blue", weight=5, opacity=0.7).add_to(m)

    # Add markers for start and end
    folium.Marker(location=start_coords, popup="Start", icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(location=end_coords, popup="Destination", icon=folium.Icon(color='red')).add_to(m)

    return m

In [13]:
start = get_current_location()
end = get_destination("Da Lat")

route, distance = calculate_route(start, end)
display(plot_route(route, start, end))

Detected current location: Hanoi, Hanoi, VN
Destination location: Thành phố Đà Lạt, Tỉnh Lâm Đồng, Việt Nam
